In [18]:
import pandas as pd
import numpy as np
import glob
import matplotlib.pyplot as plt

In [19]:
def avg_last_execution(files_path, drift_detector, generator):
    
    results = []
    for file in files_path:
        #print(file)
        df = pd.read_csv(file)
        results.append(df.iloc[-1]['classifications correct (percent)'])
        #print(df.iloc[-1]['classifications correct (percent)'])
    
    tuple_results = tuple([np.mean(results), np.std(results), drift_detector, generator])
   
    #print(f'avg of all last execution: {np.mean(median)}')
    return tuple_results 

In [20]:
drift_detectors_params = [
                              {"id": "DDM", "drift_name": "DDM", "params": ""},    \
                              {"id": "EDDM", "drift_name": "EDDM", "params": ""},   \
                              {"id": "ADWIN", "drift_name": "ADWINChangeDetector", "params": ""},  \
                              {"id": "ECDD", "drift_name": "EWMAChartDM", "params": ""},   \
                              {"id": "STEPD", "drift_name": "STEPD", "params": ""},  \
                              {"id": "SeqDrift2", "drift_name": "SeqDrift2ChangeDetector", "params": ""},  \
                              {"id": "SEED", "drift_name": "SEEDChangeDetector", "params": ""},   \
                              {"id": "HDDM_A_Test", "drift_name": "HDDM_A_Test", "params": ""},  \
                              {"id": "HDDM_W_Test", "drift_name": "HDDM_W_Test", "params": ""},  \
                              #{"id": "FHDDM", "drift_name": "FHDDM", "params": ""},  \
                              #{"id": "FTDD", "drift_name": "FTDD", "params": ""},   \
                              {"id": "RDDM_30", "drift_name": "RDDM", "params": "-n 30 -w 2 -o 3"}, \
                              {"id": "RDDM","drift_name": "RDDM", "params": ""},   \
                              #{"id": "WSTD","drift_name": "WSTD", "params": ""}
                        ]

## Ranking Gradual HoeffdingTree

In [21]:
classificator = "trees.HoeffdingTree"
data_stream = [{"data_size": 10000, "drift_position":  [2000, 2000, 2000, 2000]},   \
                {"data_size": 20000, "drift_position": [4000, 4000, 4000, 4000]},    \
                {"data_size": 50000, "drift_position": [10000, 10000, 10000, 10000]}, \
                {"data_size": 100000, "drift_position":[20000, 20000, 20000, 20000]}, \
                {"data_size": 500000, "drift_position":  [100000, 100000, 100000, 100000]}, \
                {"data_size": 1000000, "drift_position":  [200000, 200000, 200000, 200000]}, \
                {"data_size": 2000000, "drift_position": [400000, 400000, 400000, 400000]} 
               ]
type_drift = "Gradual"
dict_drift_type = {"gradual_agraw1": "AGRAW1", "gradual_agraw2": "AGRAW2", "gradual_led": "LED", 
                    "gradual_mixed": "MIXED", "gradual_randomRBF": "RANDOM_RBF", "gradual_sine": "SINE", "gradual_waveform": "WAVEFORM"}

df_result_pivot_rank = pd.DataFrame()
for data_stream_key in data_stream:
    df_final_result = pd.DataFrame()
    for drift_key, drift_value in dict_drift_type.items():    

        tuple_list_result = list()
        for drift in drift_detectors_params:
            files_path = glob.glob(f'{drift_key}/{classificator}_{drift["id"]}_{data_stream_key["data_size"]}_*_{drift_value}_{type_drift}.csv')
            #print(files_path)
            tuple_result = avg_last_execution(files_path, drift["id"], drift_value)
            tuple_list_result.append(tuple_result)
                            
        df_result = pd.DataFrame(tuple_list_result, columns=['avg', 'std', 'drift_detector', 'generator'])
        df_final_result = pd.concat([df_final_result, df_result])

    df_result_pivot = df_final_result.reset_index().pivot(index="generator", columns="drift_detector", values="avg")
    df_result_pivot_rank = pd.concat([df_result_pivot_rank, df_result_pivot.rank(1,ascending=False, method='first')])


df_result_pivot_rank.describe()

drift_detector,ADWIN,DDM,ECDD,EDDM,HDDM_A_Test,HDDM_W_Test,RDDM,RDDM_30,SEED,STEPD,SeqDrift2
count,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.00000,49.000000
mean,3.653061,2.897959,10.224490,6.000000,4.367347,4.673469,4.306122,4.387755,9.979592,8.00000,7.510204
std,2.268911,1.939125,0.918906,2.835783,1.364641,2.427170,2.311241,1.835033,1.613611,2.01039,2.246501
min,1.000000,1.000000,8.000000,1.000000,2.000000,1.000000,1.000000,1.000000,2.000000,1.00000,2.000000
25%,2.000000,1.000000,10.000000,4.000000,3.000000,2.000000,2.000000,3.000000,10.000000,8.00000,7.000000
50%,3.000000,2.000000,11.000000,7.000000,4.000000,5.000000,4.000000,4.000000,10.000000,9.00000,8.000000
75%,5.000000,4.000000,11.000000,8.000000,5.000000,6.000000,6.000000,6.000000,11.000000,9.00000,10.000000
max,9.000000,7.000000,11.000000,11.000000,8.000000,9.000000,8.000000,9.000000,11.000000,10.00000,10.000000


## Ranking Abrupt HoeffdingTree

In [22]:
classificator = "trees.HoeffdingTree"
data_stream = [{"data_size": 10000, "drift_position":  [2000, 2000, 2000, 2000]},   \
                {"data_size": 20000, "drift_position": [4000, 4000, 4000, 4000]},    \
                {"data_size": 50000, "drift_position": [10000, 10000, 10000, 10000]}, \
                {"data_size": 100000, "drift_position":[20000, 20000, 20000, 20000]}, \
                {"data_size": 500000, "drift_position":  [100000, 100000, 100000, 100000]}, \
                {"data_size": 1000000, "drift_position":  [200000, 200000, 200000, 200000]}, \
                {"data_size": 2000000, "drift_position": [400000, 400000, 400000, 400000]} 
               ]
type_drift = "Abrupt"
dict_drift_type = {"abrupt_agraw1": "AGRAW1", "abrupt_agraw2": "AGRAW2", "abrupt_led": "LED", "abrupt_mixed": "MIXED", "abrupt_randomRBF": "RANDOM_RBF",  
                    "abrupt_sine": "SINE", "abrupt_waveform": "WAVEFORM"}


df_result_pivot_rank = pd.DataFrame()
for data_stream_key in data_stream:
    df_final_result = pd.DataFrame()
    for drift_key, drift_value in dict_drift_type.items():    

        tuple_list_result = list()
        for drift in drift_detectors_params:
            files_path = glob.glob(f'{drift_key}/{classificator}_{drift["id"]}_{data_stream_key["data_size"]}_*_{drift_value}_{type_drift}.csv')
            #print(files_path)
            tuple_result = avg_last_execution(files_path, drift["id"], drift_value)
            tuple_list_result.append(tuple_result)
                            
        df_result = pd.DataFrame(tuple_list_result, columns=['avg', 'std', 'drift_detector', 'generator'])
        df_final_result = pd.concat([df_final_result, df_result])

    df_result_pivot = df_final_result.reset_index().pivot(index="generator", columns="drift_detector", values="avg")
    df_result_pivot_rank = pd.concat([df_result_pivot_rank, df_result_pivot.rank(1,ascending=False, method='first')])


df_result_pivot_rank.describe()

drift_detector,ADWIN,DDM,ECDD,EDDM,HDDM_A_Test,HDDM_W_Test,RDDM,RDDM_30,SEED,STEPD,SeqDrift2
count,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000
mean,3.591837,4.183673,10.102041,6.612245,2.897959,4.081633,4.469388,5.102041,9.693878,7.326531,7.938776
std,2.299623,2.288317,1.065460,2.841774,1.245741,2.856548,2.042283,1.310930,1.516631,2.486528,2.294996
min,1.000000,1.000000,7.000000,1.000000,1.000000,1.000000,1.000000,2.000000,5.000000,1.000000,2.000000
25%,2.000000,2.000000,9.000000,5.000000,2.000000,1.000000,3.000000,4.000000,9.000000,7.000000,7.000000
50%,3.000000,4.000000,10.000000,7.000000,3.000000,4.000000,5.000000,5.000000,10.000000,8.000000,8.000000
75%,5.000000,6.000000,11.000000,9.000000,4.000000,7.000000,6.000000,6.000000,11.000000,9.000000,10.000000
max,9.000000,10.000000,11.000000,11.000000,5.000000,9.000000,8.000000,8.000000,11.000000,10.000000,11.000000


## Ranking Gradual NaiveBayes

In [23]:
classificator = "bayes.NaiveBayes"
data_stream = [{"data_size": 10000, "drift_position":  [2000, 2000, 2000, 2000]},   \
                {"data_size": 20000, "drift_position": [4000, 4000, 4000, 4000]},    \
                {"data_size": 50000, "drift_position": [10000, 10000, 10000, 10000]}, \
                {"data_size": 100000, "drift_position":[20000, 20000, 20000, 20000]}, \
                {"data_size": 500000, "drift_position":  [100000, 100000, 100000, 100000]}, \
                {"data_size": 1000000, "drift_position":  [200000, 200000, 200000, 200000]}, \
                {"data_size": 2000000, "drift_position": [400000, 400000, 400000, 400000]} 
               ]

type_drift = "Gradual"
dict_drift_type = {"gradual_agraw1": "AGRAW1", "gradual_agraw2": "AGRAW2", "gradual_led": "LED", 
                    "gradual_mixed": "MIXED", "gradual_randomRBF": "RANDOM_RBF", "gradual_sine": "SINE", "gradual_waveform": "WAVEFORM"}

df_result_pivot_rank = pd.DataFrame()
for data_stream_key in data_stream:
    df_final_result = pd.DataFrame()
    for drift_key, drift_value in dict_drift_type.items():    

        tuple_list_result = list()
        for drift in drift_detectors_params:
            files_path = glob.glob(f'{drift_key}/{classificator}_{drift["id"]}_{data_stream_key["data_size"]}_*_{drift_value}_{type_drift}.csv')
            #print(files_path)
            tuple_result = avg_last_execution(files_path, drift["id"], drift_value)
            tuple_list_result.append(tuple_result)
                            
        df_result = pd.DataFrame(tuple_list_result, columns=['avg', 'std', 'drift_detector', 'generator'])
        df_final_result = pd.concat([df_final_result, df_result])

    df_result_pivot = df_final_result.reset_index().pivot(index="generator", columns="drift_detector", values="avg")
    df_result_pivot_rank = pd.concat([df_result_pivot_rank, df_result_pivot.rank(1,ascending=False, method='first')])


df_result_pivot_rank.describe()

drift_detector,ADWIN,DDM,ECDD,EDDM,HDDM_A_Test,HDDM_W_Test,RDDM,RDDM_30,SEED,STEPD,SeqDrift2
count,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000
mean,2.979592,5.571429,10.061224,7.571429,4.061224,5.551020,3.367347,4.102041,8.122449,8.346939,6.265306
std,2.135901,3.006936,0.987593,3.034524,1.477726,2.508828,2.472295,1.782378,2.513569,1.283729,2.970518
min,1.000000,1.000000,8.000000,2.000000,1.000000,2.000000,1.000000,1.000000,4.000000,5.000000,1.000000
25%,1.000000,2.000000,9.000000,5.000000,3.000000,3.000000,1.000000,3.000000,6.000000,8.000000,4.000000
50%,2.000000,6.000000,10.000000,9.000000,4.000000,6.000000,3.000000,4.000000,8.000000,9.000000,6.000000
75%,5.000000,8.000000,11.000000,10.000000,5.000000,8.000000,5.000000,6.000000,11.000000,9.000000,9.000000
max,9.000000,11.000000,11.000000,11.000000,7.000000,9.000000,8.000000,8.000000,11.000000,10.000000,11.000000


## Ranking Abrup NaiveBayes

In [24]:
classificator = "bayes.NaiveBayes"
data_stream = [{"data_size": 10000, "drift_position":  [2000, 2000, 2000, 2000]},   \
                {"data_size": 20000, "drift_position": [4000, 4000, 4000, 4000]},    \
                {"data_size": 50000, "drift_position": [10000, 10000, 10000, 10000]}, \
                {"data_size": 100000, "drift_position":[20000, 20000, 20000, 20000]}, \
                {"data_size": 500000, "drift_position":  [100000, 100000, 100000, 100000]}, \
                {"data_size": 1000000, "drift_position":  [200000, 200000, 200000, 200000]}, \
                {"data_size": 2000000, "drift_position": [400000, 400000, 400000, 400000]} 
               ]

type_drift = "Abrupt"
dict_drift_type = {"abrupt_agraw1": "AGRAW1", "abrupt_agraw2": "AGRAW2", "abrupt_led": "LED", "abrupt_mixed": "MIXED", "abrupt_randomRBF": "RANDOM_RBF",  
                    "abrupt_sine": "SINE", "abrupt_waveform": "WAVEFORM"}

df_result_pivot_rank = pd.DataFrame()
for data_stream_key in data_stream:
    df_final_result = pd.DataFrame()
    for drift_key, drift_value in dict_drift_type.items():    

        tuple_list_result = list()
        for drift in drift_detectors_params:
            files_path = glob.glob(f'{drift_key}/{classificator}_{drift["id"]}_{data_stream_key["data_size"]}_*_{drift_value}_{type_drift}.csv')
            #print(files_path)
            tuple_result = avg_last_execution(files_path, drift["id"], drift_value)
            tuple_list_result.append(tuple_result)
                            
        df_result = pd.DataFrame(tuple_list_result, columns=['avg', 'std', 'drift_detector', 'generator'])
        df_final_result = pd.concat([df_final_result, df_result])

    df_result_pivot = df_final_result.reset_index().pivot(index="generator", columns="drift_detector", values="avg")
    df_result_pivot_rank = pd.concat([df_result_pivot_rank, df_result_pivot.rank(1,ascending=False, method='first')])


df_result_pivot_rank.describe()

drift_detector,ADWIN,DDM,ECDD,EDDM,HDDM_A_Test,HDDM_W_Test,RDDM,RDDM_30,SEED,STEPD,SeqDrift2
count,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000
mean,3.040816,6.877551,9.755102,8.571429,3.265306,4.489796,3.510204,5.469388,7.346939,6.897959,6.775510
std,2.061140,3.364228,1.181994,2.449490,1.203736,3.202359,1.634555,1.582483,2.650087,2.616016,2.800783
min,1.000000,1.000000,6.000000,2.000000,1.000000,1.000000,1.000000,1.000000,4.000000,1.000000,2.000000
25%,1.000000,4.000000,9.000000,8.000000,2.000000,1.000000,3.000000,5.000000,5.000000,5.000000,5.000000
50%,2.000000,8.000000,10.000000,9.000000,3.000000,4.000000,3.000000,6.000000,7.000000,8.000000,7.000000
75%,5.000000,9.000000,11.000000,10.000000,4.000000,8.000000,4.000000,6.000000,11.000000,9.000000,10.000000
max,9.000000,11.000000,11.000000,11.000000,6.000000,9.000000,7.000000,8.000000,11.000000,10.000000,11.000000
